# 文本挖掘
情感分析系统主要包含：社交媒体数据预处理、词性标注、实体识别、情感分类、聚类分析等。

本章实验的主要目的是掌握社交媒体情感分析系统的主要模块用到的方法，即词性标注、实体识别、情感分类、聚类分析等。掌握如何使用华为云ModelArts Notebook上传数据、执行Python代码，掌握数据挖掘的一般建模流程。

本次实验使用的数据集来源于丁香园2019新型冠状病毒疫情新闻数据的局部样本，目的是进行疫情程度分析且最终对疫情进行程度分类。全体数据共有10443个样本，本实验随机抽取了258条样本作为示例。

### 配置信息填写

实验中需要将运行结果上传至OBS中，我们需要设置以下相关参数（使用自己真实的桶名和唯一ID替换掉*号）：

* BUCKET_NAME ： 自己的OBS桶名
* UNIQUE_ID : 唯一ID，填写自己的学号或者IAM子账号名称

In [1]:
BUCKET_NAME = 'XXX' 
UNIQUE_ID = 'XXX'
OBS_BASE_PATH = BUCKET_NAME

### 初始化ModelArts SDK

In [2]:
from modelarts.session import Session
session = Session(access_key='XXX',secret_key='XXX', project_id='XXX', region_name='cn-north-4')

###  准备数据

这一步准备实验所需的数据，相关数据资源请先上传并保存在OBS中，我们通过ModelArts SDK将资源下载到本地，并解压到当前目录下。解压后，当前目录包含DXYNews目录，存有数据集。

In [3]:
session.download_data(bucket_path=OBS_BASE_PATH+'/textanalysis/DXYNews.tar.gz', path='./DXYNews.tar.gz')
# 使用tar命令解压资源包
!tar xf DXYNews.tar.gz

OBSDownloadException: ('InvalidBucketName', 'The specified bucket is not valid.')

### 导入基本工具库
执行下面方框中的这段代码，可以导入本次实验中使用的Python开发基本工具库。

numpy是数据分处理工具，pandas是文件读取和数据处理工具，jieba是中文预处理工具（本实验用到了posseg用于词性标注）。

此段代码只是引入Python包，无回显（代码执行输出）。

In [4]:
pip install jieba

You are using pip version 9.0.1, however version 21.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
# import same usefull libraries
import numpy as np
import pandas as pd
import jieba
import jieba.posseg as psg
import jieba.analyse

### 数据读取
使用pandas库导入新闻数据，并打印前5条数据

In [6]:
#数据读取
df = pd.read_csv(r"DXYNews/DXYNews-test.csv",encoding="gbk")
df.head()

FileNotFoundError: [Errno 2] File b'DXYNews/DXYNews-test.csv' does not exist: b'DXYNews/DXYNews-test.csv'

### 词性标注
本实验使用jieba库用于中文分词。请在中文分词前根据任务需要进行预处理（具体做法参考Lab4），如，去掉空格和非汉字字符。

In [7]:
data = df['summary'].values[:5]
label = df['category'].values[:5]
part_of_speech = []
for line in data:
    seg_list = psg.cut(line)
    result = " ".join(["{0}/{1}".format(w, t) for w, t in seg_list])
    part_of_speech.append(result)
part_of_speech

NameError: name 'df' is not defined

### 保存结果至OBS

我们将词性标注结果保存到文本文件中，并上传到OBS，以便后面实验课和大作业使用。

#### 写入本地文件

将结果写入到文本文件中。会打印成功保存的信息。

In [8]:
import os
if not os.path.exists('results'):
    os.mkdir('results') # 创建本地保存路径
#字典中的key值即为csv中列名
id = df['id'].values[:5]
dataframe = pd.DataFrame({'part_of_speech':part_of_speech,'id':id})
#将DataFrame存储为csv,index表示是否显示行名，default=True
dataframe.to_csv("./results/part_of_speech_results.csv",index=False,sep=',') 
print('Successfully saved!')

NameError: name 'df' is not defined

#### 上传文件至OBS

使用ModelArts SDK上传本地文件至OBS。可以看到上传成功的日志。

In [9]:
session.upload_data(bucket_path=OBS_BASE_PATH+'/textanalysis/results/', path='./results/part_of_speech_results.csv')

Exception: ('InvalidBucketName', 'The specified bucket is not valid.')

你的OBS存储路径下，results目录下，有词性标注文件part_of_speech_results.csv'。

### 实体识别
本实验使用jieba库用于中文分词。请在中文分词前根据任务需要进行预处理（具体做法参考Lab4），如，去掉空格和非汉字字符。

In [10]:
import jieba.analyse
# 使用jieba进行词性切分，allowPOS指定允许的词性，这里选择名词n和地名ns
data = df['summary'].values[:5]
label = df['category'].values[:5]
for line in data:
    kw = jieba.analyse.extract_tags(line, topK=10, withWeight=True, allowPOS=('n', 'ns'))
    print(kw)
    for item in kw:
        print(item[0], item[1])

NameError: name 'df' is not defined

In [11]:
data = df['summary'].values[:5]
label = df['category'].values[:5]
for line in data:
    kw = jieba.analyse.textrank(line,topK=10,withWeight=True,allowPOS=('ns','n'))
    print(kw)
    for item in kw:
        print(item[0],item[1])

NameError: name 'df' is not defined

请将实体识别结果保存到文本文件中，并上传到OBS，以便后面实验课和大作业使用，可用于关键词提取。

### 文本聚类
通过实践，掌握数据分析挖掘的基本流程，及通过聚类实现群体细分和通过PCA降维实现主成分分析的基本思想，并结合实践操作加深理论的理解和认识，为实际的业务应用奠定理论和技术基础。

本实验遵从数据挖掘的一般流程，首先对数据进行读取，常规的探索后，进行数据降维操作，随后直接选择sklearn模块中的KMeans算法进行建模，通过轮廓系数对KMeans算法进行K值调优，选择最优的K值进行模型训练，最后通过统计分析对聚类结果进行画像分析，从而明确每一类的特征。

### 导入基本工具库
本实验使用到的框架主要包括numpy，pandas，scikit-learn，matplotlib，seaborn库。scikit-learn库是Python的机器学习库，提供一些常用的机器学习算法模型及模型参数优化功能；numpy，pandas库是Python中结构化数据处理的库，主要用于结构化数据的统计分析及操作；matplotlib，seaborn主要用于数据分析过程的可视化展示。

In [12]:
#加载Python库
#加载绘图模块
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA
import seaborn as sns
sns.set_style(style="darkgrid")

### 数据读取
使用pandas库导入新闻数据，并打印前2条数据

In [13]:
#数据读取
df = pd.read_csv(r"DXYNews/DXYNews-test.csv",encoding="gbk")
df.head(2)

FileNotFoundError: [Errno 2] File b'DXYNews/DXYNews-test.csv' does not exist: b'DXYNews/DXYNews-test.csv'

### 数据标准化
通过如下代码，实现文本数据集的数值向量转化

In [14]:
#设置停用词，构建词频矩阵
stopwords = []
for stopword in open('DXYNews/stopwords.txt','r'):
    stopwords.append(stopword.strip())
def tokenizer(sentense):
    words = []
    cut = jieba.cut(sentense)
    for word in cut:
        words.append(word)
    return words
count = CountVectorizer(tokenizer=tokenizer, stop_words=list(stopwords))
countvector = count.fit_transform(df['summary']).toarray()
countvector

FileNotFoundError: [Errno 2] No such file or directory: 'DXYNews/stopwords.txt'

### 数据降维
通过降维实现主要特征提取，为后续的聚类提供数据准备。

In [15]:
#此处的主成分维度我们人为设定为2，对于属性较少的数据集，属于常规会选择的维度数，后面也会看到，这个也是出于可以可视化的需求
pca = PCA(n_components = 2)
#将设置了维数的模型作用到标准化后的数据集并输出查看
newvector = pca.fit_transform(countvector)
newvector_frame = pd.DataFrame(newvector,columns=['pca_1','pca_2'])
newvector_frame.head(5)

NameError: name 'countvector' is not defined

### 降维后的结果可视化

2维示例图

In [16]:
#画图 
kind = np.unique(df['category'].values)
#降维后的结果
plt.figure(figsize=(8, 8))
for i,c,m in zip(range(len(kind)),['r','b','g','y'],['o','^','>','<']):
    index = df[df['category']==kind[i]].index
    x = newvector[index,0]
    y = newvector[index,1]
    plt.scatter(x,y,c=c,marker=m,label=kind[i])
plt.legend()
plt.xlabel('pca_1')
plt.ylabel('pca_2')

NameError: name 'df' is not defined

### K-Means聚类模型训练
在降维的数据上使用K-means聚类算法将数据聚成3类，这里的3类是事先人工指定的划分群数，通常是结合后续的业务需要或是结合业务专家的建议凭经验指定，或者是指定不同的K值后比较得来。

#### 使用PCA进行数据降维，降为3维
通过降维实现主要特征提取，为后续的聚类提供数据准备。

In [17]:
#此处的主成分维度我们人为设定为3，对于属性较少的数据集，属于常规会选择的维度数，后面也会看到，这个也是出于可以可视化的需求
new_pca = PCA(n_components=3)
#将设置了维数的模型作用到标准化后的数据集并输出查看
X_pca = new_pca.fit_transform(countvector)
X_pca_frame = pd.DataFrame(X_pca,columns=['pca_1','pca_2','pca_3'])
X_pca_frame.head(5)

NameError: name 'countvector' is not defined

#### 训练模型

In [18]:
#K-means聚类建模
#运行KMeans聚类算法
from sklearn.cluster import KMeans
#此处指定K=12
est = KMeans(n_clusters=3)
est.fit(X_pca)
#获取数据标签值
kmeans_clustering_labels = pd.DataFrame(est.labels_,columns=['cluster'])
#将聚类结果与降维特征数据进行拼接
X_pca_frame = pd.concat([X_pca_frame,kmeans_clustering_labels], axis=1) 
X_pca_frame.head(5)

NameError: name 'X_pca' is not defined

### 聚类结果分析

In [19]:
 #绘图
from mpl_toolkits.mplot3d import Axes3D
cluster_1_color = {0:'red',1:'green',2:'blue',3:'yellow',4:'cyan',5:'black',6:'magenta', 7:'#fff0f5', 8:'#ffdab9',9:'#ffa500' }
colors_clustered_data = X_pca_frame['cluster'].map(cluster_1_color)
fig_clustered_data = plt.figure()
ax_clustered_data = fig_clustered_data.add_subplot(111, projection='3d')
ax_clustered_data.scatter(X_pca_frame['pca_1'].values,X_pca_frame['pca_2'].values,X_pca_frame['pca_3'].values,c=colors_clustered_data)
ax_clustered_data.set_xlabel('Component 1')
ax_clustered_data.set_ylabel('Component 2')
ax_clustered_data.set_zlabel('Component 3')

NameError: name 'X_pca_frame' is not defined

<p style="font-family: Arial; font-size:1.4em;color:red;">请保存聚类结果到本地文件并上传到OBS。</p>